# Bibliotecas

In [122]:
import math
from itertools import combinations

# Datos

In [123]:
# --- Data ---
circles = [
    ((9.5, 30.5), 3.5),
    # ((11, 30.5), 3.5),     # Quitar para prueba sin intersecciones
    ((18, 24), 5),  
    ((4, 23), 3),
    # ((4, 17), 4),          # Quitar para prueba sin intersecciones
    # ((7, 17), 4.5),        # Quitar para prueba sin intersecciones
    ((12.5,17), 3),
    # ((18, 15), 6),         # Quitar para prueba sin intersecciones
    # ((27,19),3),           # Quitar para prueba sin intersecciones
    ((30,17),3),
    ((4, 11), 3),
    # ((12, 9), 2),          # Quitar para prueba sin intersecciones
    ((16,9),3),    
    # ((6, 4),  3),          # Quitar para prueba sin intersecciones
    ((7.5, 4), 2.5)
    # ((18, 5), 5)           # Quitar para prueba sin intersecciones
        ]

points = [
    (28, 18), 
    (9, 17), 
    (18, 9.5), 
    (18, 20), 
    (11, 5), 
    (14, 30.5),
    (40,40)
        ]

# Funciones

## Funciones de Q

In [124]:
def build_Q(circles, points):
    Q = []
    # 1) Input points
    for idx, (x, y) in enumerate(points, start=1):
        Q.append((x, y, "input", {idx}))
    # 2) Left endpoints of circles
    for idx, ((cx, cy), r) in enumerate(circles, start=1):
        Q.append((cx - r, cy, "left", {idx}))
    # 3) Order the list Q by x, then by y
    return order_Q(Q)

In [125]:
def order_Q(Q):
    Q.sort(key=lambda t: (t[0], t[1]))
    return Q

In [126]:
def event_in_Q(Q, event, tol=1e-9):
    """
    Regresa True si existe en Q un evento con mismo tipo e ids y con (x,y)
    a distancia <= tol. De lo contrario False.
    event debe ser (x, y, tipo, ids_set)
    """
    x, y, etype, eids = event
    for (qx, qy, qtype, qids) in Q:
        if qtype != etype:
            continue
        if qids != eids:
            continue
        if abs(qx - x) <= tol and abs(qy - y) <= tol:
            return True
    return False


def add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True):
    """
    Agrega a Q los eventos de `nuevos_eventos` (tuplas estilo Q: (x, y, "intersection", {i,j}))
    que no existan ya en Q (según `event_in_Q`).
    Modifica Q in place. Regresa el número de eventos agregados.
    """
    agregados = 0
    for ev in nuevos_eventos:
        # Validación básica de estructura
        if not (isinstance(ev, (list, tuple)) and len(ev) == 4 and isinstance(ev[3], set)):
            continue
        if not event_in_Q(Q, ev, tol=tol):
            Q.append(ev)
            agregados += 1

    # if agregados and keep_sorted:
    #     # Ordena por (x, y) como tu order_Q
    #     Q.sort(key=lambda t: (t[0], t[1]))
    return agregados


In [127]:
Q = build_Q(circles, points)
print(Q)

[(1, 11, 'left', {6}), (1, 23, 'left', {3}), (5.0, 4, 'left', {8}), (6.0, 30.5, 'left', {1}), (9, 17, 'input', {2}), (9.5, 17, 'left', {4}), (11, 5, 'input', {5}), (13, 9, 'left', {7}), (13, 24, 'left', {2}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (27, 17, 'left', {5}), (28, 18, 'input', {1}), (40, 40, 'input', {7})]


In [128]:
def print_Q(Q, label="Q"):
    """
    Imprime Q con el formato:

    Q:  [
        (1, 11, 'left', {6}),
        (1, 23, 'left', {3}),
        ...
    ]

    Devuelve el texto por si quieres guardarlo o registrarlo.
    """

    def fmt_set(s):
        # Acepta set/frozenset/iterable; garantiza orden y el caso set()
        s = set(s)
        if not s:
            return "set()"
        return "{" + ", ".join(str(x) for x in sorted(s)) + "}"

    def fmt_val(v):
        if isinstance(v, (set, frozenset)):
            return fmt_set(v)
        # repr conserva comillas para strings y decimales como 13.0 si vienen así
        return repr(v)

    def fmt_tuple(t):
        return "(" + ", ".join(fmt_val(v) for v in t) + ")"

    lines = [f"{label}:  ["]  # Ojo: dos espacios tras los dos puntos
    for i, item in enumerate(Q):
        comma = "," if i < len(Q) - 1 else ""
        lines.append("    " + fmt_tuple(item) + comma)
    lines.append("]")

    text = "\n".join(lines)
    print(text)
    return text


In [129]:
print_Q(Q, label="Q")

Q:  [
    (1, 11, 'left', {6}),
    (1, 23, 'left', {3}),
    (5.0, 4, 'left', {8}),
    (6.0, 30.5, 'left', {1}),
    (9, 17, 'input', {2}),
    (9.5, 17, 'left', {4}),
    (11, 5, 'input', {5}),
    (13, 9, 'left', {7}),
    (13, 24, 'left', {2}),
    (14, 30.5, 'input', {6}),
    (18, 9.5, 'input', {3}),
    (18, 20, 'input', {4}),
    (27, 17, 'left', {5}),
    (28, 18, 'input', {1}),
    (40, 40, 'input', {7})
]


"Q:  [\n    (1, 11, 'left', {6}),\n    (1, 23, 'left', {3}),\n    (5.0, 4, 'left', {8}),\n    (6.0, 30.5, 'left', {1}),\n    (9, 17, 'input', {2}),\n    (9.5, 17, 'left', {4}),\n    (11, 5, 'input', {5}),\n    (13, 9, 'left', {7}),\n    (13, 24, 'left', {2}),\n    (14, 30.5, 'input', {6}),\n    (18, 9.5, 'input', {3}),\n    (18, 20, 'input', {4}),\n    (27, 17, 'left', {5}),\n    (28, 18, 'input', {1}),\n    (40, 40, 'input', {7})\n]"

## Funciones de L

In [130]:
eps = 1e-9
x0 = Q[0][0] - eps

sweepL = {
    "x": x0,
    "active": set(),
    "conditions": [
        {"in": set(), "out": set(), "ineq": []}
    ]
}

In [131]:
def advance_sweep_to(sweepL, x_event, circles):
    sweepL["x"] = x_event

In [132]:
import copy

def remove_circle_from_L(L, circle_id, *, copy_result=False, keep_empty_base=True):
    """
    Elimina toda referencia a `circle_id` dentro de L:
      - Lo quita de L['active']
      - Lo quita de cada condición (sets 'in' y 'out')
      - Elimina toda inecuación de 'ineq' que haga referencia a (circle_id, 'y_low'/'y_high')

    Params:
        L (dict): estructura con claves 'x', 'active' (set), 'conditions' (lista de dicts).
        circle_id (int): id del círculo a purgar.
        copy_result (bool): si True, trabaja sobre una copia y la devuelve.
        keep_empty_base (bool): si True, si después no queda ninguna condición, deja una base vacía.

    Return:
        dict: L purgada (misma referencia si copy_result=False).
    """
    L2 = copy.deepcopy(L) if copy_result else L

    # 1) Quitar de activos
    if 'active' in L2 and isinstance(L2['active'], set):
        L2['active'].discard(circle_id)

    # 2) Limpiar condiciones
    new_conditions = []
    seen = set()  # para deduplicar condiciones idénticas

    for cond in L2.get('conditions', []):
        # obtener campos con defaults seguros
        in_set  = set(cond.get('in',  set()))
        out_set = set(cond.get('out', set()))
        ineqs   = list(cond.get('ineq', []))

        # filtrar referencias al círculo
        in_set.discard(circle_id)
        out_set.discard(circle_id)

        # eliminar inecuaciones que mencionen a este círculo
        def mentions_circle(ineq):
            # esperamos tuplas del tipo ('y', '<'|'>', (id, 'y_low'|'y_high'))
            try:
                return isinstance(ineq, tuple) and len(ineq) >= 3 \
                       and isinstance(ineq[2], tuple) and ineq[2][0] == circle_id
            except Exception:
                return False

        ineqs = [iq for iq in ineqs if not mentions_circle(iq)]

        # si quedó totalmente vacía, podemos omitirla
        if not in_set and not out_set and not ineqs:
            continue

        # deduplicación (sin alterar el orden real en new_conditions)
        cond_key = (tuple(sorted(in_set)), tuple(sorted(out_set)), tuple(sorted(ineqs)))
        if cond_key not in seen:
            new_conditions.append({'in': in_set, 'out': out_set, 'ineq': ineqs})
            seen.add(cond_key)

    # 3) Si no quedan condiciones y queremos una base vacía, añadimos una
    if not new_conditions and keep_empty_base:
        new_conditions = [{'in': set(), 'out': set(), 'ineq': []}]

    L2['conditions'] = new_conditions
    return L2


In [133]:
def print_sweepL(L, label="L"):
    """
    Imprime sweepL con el formato:
    L {'x': 13, 'active': {1, 2}, 
    'conditions': [
        {'in': set(), 'out': {1}, 'ineq': [...]},
        ...
    ]}
    Si L es str, solo la antepone con el label y la imprime.
    Devuelve el texto final por si quieres guardarlo o loguearlo.
    """
    if isinstance(L, str):
        text = f"{label} {L}"
        print(text)
        return text

    def fmt_set(s):
        if not s:
            return "set()"
        return "{" + ", ".join(str(x) for x in sorted(s)) + "}"

    def fmt_tuple(t):
        # repr para strings; números tal cual
        return "(" + ", ".join(repr(x) for x in t) + ")"

    def fmt_ineq(lst):
        if not lst:
            return "[]"
        return "[" + ", ".join(fmt_tuple(t) for t in lst) + "]"

    def fmt_condition(c):
        return (
            "{"
            f"'in': {fmt_set(c.get('in', set()))}, "
            f"'out': {fmt_set(c.get('out', set()))}, "
            f"'ineq': {fmt_ineq(c.get('ineq', []))}"
            "}"
        )

    x = L.get("x")
    active = fmt_set(L.get("active", set()))
    conditions = L.get("conditions", [])

    lines = []
    lines.append(f"{label} {{'x': {x}, 'active': {active}, ")
    lines.append("'conditions': [")
    for i, cond in enumerate(conditions):
        comma = "," if i < len(conditions) - 1 else ""
        lines.append("    " + fmt_condition(cond) + comma)
    lines.append("]}")
    text = "\n".join(lines)
    print(text)
    return text


In [134]:
print_sweepL(sweepL, label="sweepL")

sweepL {'x': 0.999999999, 'active': set(), 
'conditions': [
    {'in': set(), 'out': set(), 'ineq': []}
]}


"sweepL {'x': 0.999999999, 'active': set(), \n'conditions': [\n    {'in': set(), 'out': set(), 'ineq': []}\n]}"

## Condiciones

In [135]:
def in_circle(x, y, cx, cy, r, tolerancia=1e-12):
    """
    Evalua si un punto se encuentra dentro de un círculo.
    Regresa Verdadero o Falso, dependiendo si cumpe o no.
    Se usa una tolerancia para evitar errores por punto flotante
    """
    print("Evalua el punto (",x,",",y,") en el círculo con centro en (",cx,",",cy,") y radio ",r)
    print((x - cx)**2 + (y - cy)**2, " <= ", r**2 + tolerancia, (x - cx)**2 + (y - cy)**2 <= r**2 + tolerancia)
    return (x - cx)**2 + (y - cy)**2 <= r**2 + tolerancia

In [136]:
def circle_y_bounds_at_x(circle, x0):
    """
    Regresa los cortes (y_low, y_high) de la línea x = x0 
    con el círculo con centro en (cx, cy) y radio r
    Si la línea no intersecta al círculo regresa (None, None).
    """
    (cx, cy), r = circle
    # Calcula la distancia horizontal entre el centro del círculo
    # y el punto x0
    dx = x0 - cx
    # Si esa distancia es mayor al radio, no se intersectan
    if abs(dx) > r:
        return None, None
    # Si s[i se intersectan, calcula las intersecciones
    h = math.sqrt(max(r*r - dx*dx, 0.0))
    return cy - h, cy + h

In [137]:
# Hay 2 tipos de condiciones que puede tener una región
# con respecto a un cículo activo en la línea de barrido sweepL:
#    * Que esté dentro de ese círculo activo en sweepL
#    * Que esté fuera de ese círculo activo en sweepL
# y, si está fuera puede estar:
#    * Por encima de ese círculo
#    * Por debajo de ese círculo

def satisfies(cond, x, y, circles):
    # para evaluar si está dentro del círculo
    for cid in cond.get("in", set()):
        (cx, cy), r = circles[cid - 1]
        print("Reviso si SÍ está en el círculo")
        if not in_circle(x, y, cx, cy, r):
            return False

    # para evaluar si está fuera del circulo 
    for cid in cond.get("out", set()):
        (cx, cy), r = circles[cid - 1]
        print("Reviso si NO está en el círculo")
        if in_circle(x, y, cx, cy, r):
            return False

    # para evaluar si está por encima o por debajo
    for var, op, (cid, which) in cond.get("ineq", []):
        y_low, y_high = circle_y_bounds_at_x(circles[cid - 1], x)
        print("Evaluo yhigh y ylow")
        if y_low is None:
            # If the sweep line x doesn't intersect this circle at all,
            # this inequality is not meaningful. Be conservative and fail.
            return False
        y_ref = y_low if which == "y_low" else y_high
        if op == "<" and not (y < y_ref): return False
        if op == ">" and not (y > y_ref): return False

    return True

## Leftend point

In [138]:
def leftend_point(Q, sweepL, circle_id, circles, eps=1e-6):
    """
    Insert circle `circle_id` into the conditions model at sweepL['x'].
    We only add the logical conditions; boundaries are computed lazily in `satisfies`.
    """
    (cx, cy), r = circles[circle_id - 1]
    print("Círculo izquierdo con centro en (",cx,",",cy,") y radio ",r)
    
    # Evaluate just to the right of the left endpoint to avoid tangency
    x_prime = sweepL["x"] + eps
    dx = x_prime - cx
    if abs(dx) > r + 1e-15:
        # numerically outside the circle's vertical span at x', nothing to split
        sweepL.setdefault("active", set()).add(circle_id)
        return Q, sweepL

    # Find the single condition that contains (x_prime, cy) *before* inserting the circle
    conds = sweepL["conditions"]
    print("Conditions",sweepL)
    k = None
    for i, cond in enumerate(conds):
        if satisfies(cond, x_prime, cy, circles):
            k = i
            break
    if k is None:
        k = len(conds) - 1  # fallback

    base = conds[k]
    print("base", base)
    base_in   = set(base.get("in",  set()))
    base_out  = set(base.get("out", set()))
    base_ineq = list(base.get("ineq", []))

    # Replace with three regions using symbolic inequalities tied to (circle_id, "y_low"/"y_high")
    down = {
        "in":   set(base_in),
        "out":  set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", "<", (circle_id, "y_low"))],
    }
    middle = {
        "in":   set(base_in) | {circle_id},
        "out":  set(base_out),
        "ineq": list(base_ineq),
    }
    up = {
        "in":   set(base_in),
        "out":  set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", ">", (circle_id, "y_high"))],
    }

    print("down", down)
    print("middle", middle)
    print("up", up)

    print("Revisar intersección arriba: ", mark_intersection_needed(up))
    if mark_intersection_needed(up):
        print(intersections_from_condition(up, circles))
        nuevos_eventos = intersections_from_condition(up, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)
    print("Revisar intersección abajo: ", mark_intersection_needed(down))
    if mark_intersection_needed(down):
        print(intersections_from_condition(down, circles))
        nuevos_eventos = intersections_from_condition(down, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    Q.append((cx + r, cy, "right", {circle_id}))
    order_Q(Q)
    print_Q(Q, label="Q")

    sweepL["conditions"] = conds[:k] + [down, middle, up] + conds[k+1:]
    sweepL.setdefault("active", set()).add(circle_id)

    print_sweepL(sweepL, label="L")
    return Q, sweepL

In [139]:
def mark_intersection_needed(cond, var="y"):
    """
    Return 'check for intersection' if `cond["ineq"]` contains BOTH '<' and '>' 
    inequalities for the given variable (default 'y'). Supports ANY number of inequalities.
    Otherwise return None.
    """
    lt = gt = False
    for item in cond.get("ineq", []):
        if not isinstance(item, (list, tuple)) or len(item) != 3:
            continue
        v, op, _ = item
        if v != var:
            continue
        if op == "<":
            lt = True
        elif op == ">":
            gt = True
        if lt and gt:
            return True
    return None


# Example:
down = {'in': {11}, 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}
up   = {'in': {11}, 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}
print(mark_intersection_needed(up))  # -> "check for intersection"
print(mark_intersection_needed(down))

True
None


In [140]:
def circle_circle_intersections(c1, c2, tol=1e-12):
    (x0, y0), r0 = c1
    (x1, y1), r1 = c2
    dx, dy = x1 - x0, y1 - y0
    d = math.hypot(dx, dy)

    # no solutions or infinite solutions (coincident) -> return none
    if d > r0 + r1 + tol:         # separate
        return []
    if d < abs(r0 - r1) - tol:    # contained
        return []
    if d < tol and abs(r0 - r1) < tol:  # coincident
        return []

    # base point along the line of centers
    a = (r0*r0 - r1*r1 + d*d) / (2*d)
    h2 = r0*r0 - a*a
    if h2 < -tol:
        return []
    h = math.sqrt(max(h2, 0.0))

    xm = x0 + a * dx / d
    ym = y0 + a * dy / d

    if h <= tol:  # tangent (one point)
        return [(xm, ym)]

    # two intersection points
    rx = -dy * (h / d)
    ry =  dx * (h / d)
    return [(xm + rx, ym + ry), (xm - rx, ym - ry)]

def intersections_from_condition(cond, circles, var="y"):
    """
    Collect circle IDs from cond['ineq'] (for `var`), compute pairwise intersections,
    and return Q-style tuples: (x, y, "intersection", {cid1, cid2}).
    """
    circle_ids = sorted({cid for v, _, (cid, _) in cond.get("ineq", []) if v == var})
    if len(circle_ids) < 2:
        return []

    events = []
    for i, j in combinations(circle_ids, 2):
        pts = circle_circle_intersections(circles[i - 1], circles[j - 1])
        for (x, y) in pts:
            events.append((x, y, "intersection", {i, j}))
    return events

# # Example:
# up = {'in': {11}, 'out': {11, 5}, 'ineq': [('y','<',(5,'y_low')), ('y','>',(11,'y_high'))]}
# print(intersections_from_condition(up, circles))


In [141]:
# split_segment_for_new_circle(sweepL, circle_id=5, circles=circles, eps=1e-6)
leftend_point(Q, sweepL, circle_id=5, circles=circles, eps=1e-6)

Círculo izquierdo con centro en ( 30 , 17 ) y radio  3


([(1, 11, 'left', {6}),
  (1, 23, 'left', {3}),
  (5.0, 4, 'left', {8}),
  (6.0, 30.5, 'left', {1}),
  (9, 17, 'input', {2}),
  (9.5, 17, 'left', {4}),
  (11, 5, 'input', {5}),
  (13, 9, 'left', {7}),
  (13, 24, 'left', {2}),
  (14, 30.5, 'input', {6}),
  (18, 9.5, 'input', {3}),
  (18, 20, 'input', {4}),
  (27, 17, 'left', {5}),
  (28, 18, 'input', {1}),
  (40, 40, 'input', {7})],
 {'x': 0.999999999,
  'active': {5},
  'conditions': [{'in': set(), 'out': set(), 'ineq': []}]})

In [142]:
# # Advance to circle 11's left endpoint
# (cx, cy), r = circles[11 - 1]
# advance_sweep_to(sweepL, cx - r, circles)

# leftend_point(Q, sweepL, circle_id=11, circles=circles, eps=1e-6)

In [143]:
# sweepL

## Rightendpoint

In [144]:
def rightend_point(Q, sweepL, circle_id, circles, eps=1e-6):
    """
    Procesa el extremo derecho del círculo `circle_id` en la posición sweepL['x'].
    - Busca la región 'middle' (condición con circle_id en 'in') que contiene (x - eps, cy)
    - Toma sus vecinas 'down' (k-1) y 'up' (k+1)
    - Opcional: detecta intersecciones en up/down (igual que en leftend_point)
    - Elimina la región 'middle' y fusiona up y down removiendo referencias al círculo
    - No agrega evento 'right' a Q
    """
    (cx, cy), r = circles[circle_id - 1]
    print("Círculo derecho con centro en (", cx, ",", cy, ") y radio ", r)

    x_prime = sweepL["x"] - eps  # evaluar justo antes del extremo derecho
    conds = sweepL["conditions"]
    print_sweepL(sweepL, label="Conditions")

    # 1) localizar la región 'middle' (con circle_id en 'in') que contiene (x', cy)
    k = None
    for i, cond in enumerate(conds):
        if circle_id in cond.get("in", set()) and satisfies(cond, x_prime, cy, circles):
            k = i
            break

    if k is None:
        # No se encontró región media; desactivar y salir
        sweepL.setdefault("active", set()).discard(circle_id)
        print("No se encontró región con el círculo en 'in' para (x', cy); no se fusiona.")
        return Q, sweepL

    # Deben existir vecinos arriba y abajo
    if k - 1 < 0 or k + 1 >= len(conds):
        sweepL.setdefault("active", set()).discard(circle_id)
        print("No hay vecinos up/down contiguos; no se fusiona.")
        return Q, sweepL

    down  = conds[k - 1]
    mid   = conds[k]
    up    = conds[k + 1]

    print("down", down)
    print("middle", mid)
    print("up", up)

    # 2) Igual que en leftend_point: revisar intersecciones en up y down
    if mark_intersection_needed(up):
        print("Revisar intersección arriba:", intersections_from_condition(up, circles))
        nuevos_eventos = intersections_from_condition(up, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    if mark_intersection_needed(down):
        print("Revisar intersección abajo:", intersections_from_condition(down, circles))
        nuevos_eventos = intersections_from_condition(down, circles)
        add_unique_events(Q, nuevos_eventos, tol=1e-9, keep_sorted=True)

    # 3) Fusionar up y down en una región base, quitando referencias a circle_id
    def _sin_ineq_del_circulo(ineqs, cid):
        # elimina desigualdades que referencien a este círculo
        filtradas = []
        for t in ineqs:
            if not (isinstance(t, (list, tuple)) and len(t) == 3):
                filtradas.append(t); continue
            var, op, ref = t
            if var == "y" and isinstance(ref, tuple) and ref[0] == cid:
                continue  # quitarla
            filtradas.append(t)
        return filtradas

    # in/out base: intersección de ambos vecinos (y quitando el círculo de 'out')
    base_in  = set(down.get("in", set())) & set(up.get("in", set()))
    base_out = (set(down.get("out", set())) & set(up.get("out", set()))) - {circle_id}

    # ineq base: unión de ineqs de up y down sin referencias al círculo que cierra (deduplicada)
    ineq_down = _sin_ineq_del_circulo(down.get("ineq", []), circle_id)
    ineq_up   = _sin_ineq_del_circulo(up.get("ineq", []), circle_id)

    base_ineq = []
    for t in ineq_down + ineq_up:
        if t not in base_ineq:
            base_ineq.append(t)

    merged = {"in": base_in, "out": base_out, "ineq": base_ineq}
    print("merged", merged)

    # 4) Reemplazar [down, middle, up] por [merged]
    sweepL["conditions"] = conds[:k - 1] + [merged] + conds[k + 2:]
    sweepL.setdefault("active", set()).discard(circle_id)

    sweepL = remove_circle_from_L(sweepL, circle_id)

    # print("L (tras cerrar círculo)", sweepL)
    print_sweepL(sweepL, label="L (tras cerrar círculo)")
    # Nota: NO agregamos evento 'right' a Q
    return Q, sweepL


In [145]:
# # Avanza a la coordenada del extremo derecho del círculo 11
# (cx, cy), r = circles[11 - 1]
# advance_sweep_to(sweepL, cx + r, circles)

# # Procesa el extremo derecho del círculo 11
# Q, sweepL = rightend_point(Q, sweepL, circle_id=11, circles=circles, eps=1e-6)

# # Opcional: inspeccionar estado
# print("Q:", Q)
# print("L:", sweepL)


## Input point

In [146]:
def input_point(sweepL, Ac, circles, q):
    """
    Remove from Ac any circle whose ID is in sweepL['active'] and that contains q.
    IDs in sweepL['active'] are 1-based indexes into `circles`.
    q can be (x, y) or (x, y, 'input', {...}).
    """
    if not (isinstance(q, tuple) and len(q) >= 2):
        raise ValueError("q must be a tuple like (x,y) or (x,y,'input',{...})")
    x, y = q[0], q[1]

    active_ids = set(sweepL.get("active", set()))
    if not active_ids:
        return Ac

    EPS = 1e-9  # for boundary-inclusive check
    TOL = 1e-9  # to match circles in Ac by value

    def point_in_circle(px, py, circle):
        (cx, cy), r = circle
        return (px - cx) ** 2 + (py - cy) ** 2 <= r ** 2 + EPS

    def same_circle(a, b):
        (ax, ay), ar = a
        (bx, by), br = b
        return (abs(ax - bx) <= TOL and
                abs(ay - by) <= TOL and
                abs(ar - br) <= TOL)

    to_remove = []
    for cid in active_ids:
        if 1 <= cid <= len(circles):
            c = circles[cid - 1]  # 1-based IDs
            if point_in_circle(x, y, c):
                to_remove.append(c)

    if not to_remove:
        return Ac

    return [c for c in Ac if not any(same_circle(c, r) for r in to_remove)]


In [147]:
Ac = circles
Ac

[((9.5, 30.5), 3.5),
 ((18, 24), 5),
 ((4, 23), 3),
 ((12.5, 17), 3),
 ((30, 17), 3),
 ((4, 11), 3),
 ((16, 9), 3),
 ((7.5, 4), 2.5)]

In [148]:

sweepL = {'x': 27, 'active': {4}, 'conditions': [
    {'in': set(), 'out': {4}, 'ineq': [('y', '<', (4, 'y_low'))]},
    {'in': {4}, 'out': set(), 'ineq': []},
    {'in': set(), 'out': {4}, 'ineq': [('y', '>', (4, 'y_high'))]}
]}
q = (28, 18)
input_point(sweepL, Ac, circles, q)

[((9.5, 30.5), 3.5),
 ((18, 24), 5),
 ((4, 23), 3),
 ((12.5, 17), 3),
 ((30, 17), 3),
 ((4, 11), 3),
 ((16, 9), 3),
 ((7.5, 4), 2.5)]

# Algoritmo final

In [152]:
# def algoritmo(circles, points):
#     # Inicializar variables
#     Ac = circles
#     Q = build_Q(circles, points)
#     eps = 1e-9
#     x0 = Q[0][0] - eps
#     sweepL = {
#     "x": x0,
#     "active": set(),
#     "conditions": [
#         {"in": set(), "out": set(), "ineq": []}
#         ]
#     }
#     print("Antes de iniciar:")
#     print_Q(Q, label="Q")
#     print("Línea de barrido =", sweepL)
#     print("Ac =", Ac)
#     i = 0
#     while i < len(Q):
#         print_Q(Q, label="Q")
#         x, y, kind, idx = Q[i]

#         advance_sweep_to(sweepL, x, circles)
        
#         if kind == "input":
#             print(f"Point ({x}, {y}) is an INPUT point, index {idx}")
    
#         elif kind == "left":
#             circle_idx = list(idx)[0]  # get the circle index from the set
#             print(f"Point ({x}, {y}) is a LEFT endpoint of circle {idx}")
#             Q, sweepL = leftend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)
    
#         elif kind == "right":
#             circle_idx = list(idx)[0]
#             print(f"Point ({x}, {y}) is a RIGHT endpoint of circle {idx}")
#             Q, sweepL = rightend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)

#         elif kind == "intersection":
#             print(f"Point ({x}, {y}) is a INTERSECTION point of circles {idx}")
#         i += 1

def algoritmo(circles, points):
    # Inicializar variables
    Ac = list(circles)  # copia independiente
    Q = build_Q(circles, points)
    eps = 1e-9
    x0 = Q[0][0] - eps
    sweepL = {
        "x": x0,
        "active": set(),
        "conditions": [
            {"in": set(), "out": set(), "ineq": []}
        ]
    }

    print("Antes de iniciar:")
    print_Q(Q, label="Q")
    print("Línea de barrido =", sweepL)
    print("Ac =", Ac)

    # helper para mapear círculos en Ac a IDs (1-based) respecto a 'circles'
    def ac_ids(ac_list):
        ids = set()
        for i, c in enumerate(circles, start=1):
            # igualdad estructural exacta está bien si no hay copias con redondeo distinto
            if c in ac_list:
                ids.add(i)
        return ids

    i = 0
    while i < len(Q):
        print_Q(Q, label="Q")
        x, y, kind, idx = Q[i]

        advance_sweep_to(sweepL, x, circles)

        if kind == "input":
            print(f"Point ({x}, {y}) is an INPUT point, index {idx}")
            before_ids = ac_ids(Ac)
            new_Ac = input_point(sweepL, Ac, circles, Q[i])  # usa tu función
            after_ids = ac_ids(new_Ac)
            removed = sorted(before_ids - after_ids)
            if removed:
                print(f" -> Removed from Ac (by ID): {removed}")
            else:
                print(" -> No active circle contained this input point.")
            Ac = new_Ac

        elif kind == "left":
            circle_idx = list(idx)[0]  # get the circle index from the set
            print(f"Point ({x}, {y}) is a LEFT endpoint of circle {idx}")
            Q, sweepL = leftend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)

        elif kind == "right":
            circle_idx = list(idx)[0]
            print(f"Point ({x}, {y}) is a RIGHT endpoint of circle {idx}")
            Q, sweepL = rightend_point(Q, sweepL, circle_id=circle_idx, circles=circles, eps=1e-6)

        elif kind == "intersection":
            print(f"Point ({x}, {y}) is a INTERSECTION point of circles {idx}")

        i += 1
    print("Ac: ",Ac)
    # opcional: devolver el estado final por si lo quieres usar
    return Ac, Q, sweepL



algoritmo(circles, points)

Antes de iniciar:
Q:  [
    (1, 11, 'left', {6}),
    (1, 23, 'left', {3}),
    (5.0, 4, 'left', {8}),
    (6.0, 30.5, 'left', {1}),
    (9, 17, 'input', {2}),
    (9.5, 17, 'left', {4}),
    (11, 5, 'input', {5}),
    (13, 9, 'left', {7}),
    (13, 24, 'left', {2}),
    (14, 30.5, 'input', {6}),
    (18, 9.5, 'input', {3}),
    (18, 20, 'input', {4}),
    (27, 17, 'left', {5}),
    (28, 18, 'input', {1}),
    (40, 40, 'input', {7})
]
Línea de barrido = {'x': 0.999999999, 'active': set(), 'conditions': [{'in': set(), 'out': set(), 'ineq': []}]}
Ac = [((9.5, 30.5), 3.5), ((18, 24), 5), ((4, 23), 3), ((12.5, 17), 3), ((30, 17), 3), ((4, 11), 3), ((16, 9), 3), ((7.5, 4), 2.5)]
Q:  [
    (1, 11, 'left', {6}),
    (1, 23, 'left', {3}),
    (5.0, 4, 'left', {8}),
    (6.0, 30.5, 'left', {1}),
    (9, 17, 'input', {2}),
    (9.5, 17, 'left', {4}),
    (11, 5, 'input', {5}),
    (13, 9, 'left', {7}),
    (13, 24, 'left', {2}),
    (14, 30.5, 'input', {6}),
    (18, 9.5, 'input', {3}),
    (18,

([((9.5, 30.5), 3.5),
  ((4, 23), 3),
  ((12.5, 17), 3),
  ((4, 11), 3),
  ((7.5, 4), 2.5)],
 [(1, 11, 'left', {6}),
  (1, 23, 'left', {3}),
  (5.0, 4, 'left', {8}),
  (6.0, 30.5, 'left', {1}),
  (7, 11, 'right', {6}),
  (7, 23, 'right', {3}),
  (9, 17, 'input', {2}),
  (9.5, 17, 'left', {4}),
  (10.0, 4, 'right', {8}),
  (11, 5, 'input', {5}),
  (13, 9, 'left', {7}),
  (13, 24, 'left', {2}),
  (13.0, 30.5, 'right', {1}),
  (14, 30.5, 'input', {6}),
  (15.5, 17, 'right', {4}),
  (18, 9.5, 'input', {3}),
  (18, 20, 'input', {4}),
  (19, 9, 'right', {7}),
  (23, 24, 'right', {2}),
  (27, 17, 'left', {5}),
  (28, 18, 'input', {1}),
  (33, 17, 'right', {5}),
  (40, 40, 'input', {7})],
 {'x': 40,
  'active': set(),
  'conditions': [{'in': set(), 'out': set(), 'ineq': []}]})